In [2]:
import pandas as pd
import numpy as np

In [4]:
df_train = pd.read_csv("./data/train.csv")
df_train.head()
# df_train

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
0,122881,hdb flat for sale in 866 yishun street 81,sembawang / yishun (d27),866 yishun street 81,hdb 4 rooms,NaN,1988.0,3.0,2.0,1115,...,unspecified,NaN,116.0,https://www.99.co/singapore/hdb/866-yishun-str...,1.414399,103.837196,0,yishun south,yishun,514500.0
1,259374,hdb flat for sale in 506b serangoon north aven...,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,99-year leasehold,1992.0,4.0,2.0,1575,...,unspecified,"1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbserangoon-e...,1.372597,103.875625,0,serangoon north,serangoon,995400.0
2,665422,4 bed condo for sale in meyerhouse,128 meyer road,meyerhouse,condo,freehold,2022.0,4.0,6.0,3070,...,partial,"studio, 3, 4, 5, 6 br",56.0,https://www.99.co/singapore/condos-apartments/...,1.298773,103.895798,0,mountbatten,marine parade,8485000.0
3,857699,3 bed condo for sale in leedon green,26 leedon heights,leedon green,Condo,freehold,2023.0,3.0,2.0,958,...,partial,"studio, 1, 2, 3, 4 br",638.0,https://www.99.co/singapore/condos-apartments/...,1.312364,103.803271,0,farrer court,bukit timah,2626000.0
4,216061,2 bed condo for sale in one bernam,1 bernam street,one bernam,condo,99-year leasehold,2026.0,2.0,1.0,732,...,unspecified,"studio, 1, 2, 3, 4, 5 br",351.0,https://www.99.co/singapore/condos-apartments/...,1.273959,103.843635,0,anson,downtown core,1764000.0


In [5]:
# num_beds: fill 1 with studio
df_train['num_beds'] = np.where((df_train['num_beds'].isna() & df_train['title'].str.startswith('studio ')), 
                                1, df_train['num_beds'])
# drop num_bed empty rows (5)
df_train = df_train[df_train['num_beds'].notna()]

In [6]:
def min_dist_feature_in_same_subzone (fill_in_feature, subzone, lat, lng, df):
  df_subset = df[df['subzone'] == subzone]
  df_subset = df_subset.reset_index(drop=True)
  return df.iloc[np.argmin(np.sqrt((df["lat"]-lat)**2+(df["lng"]-lng)**2))][fill_in_feature]

def fill_NA_with_nearest_record(df, empty_feature):
  df_empty = df[(df[empty_feature].isna())]
  df_empty[empty_feature] = df_empty.apply(lambda row: min_dist_feature_in_same_subzone(empty_feature, row['subzone'], row['lat'], row['lng'], df[(df[empty_feature].notna())]), axis=1)
  return df_empty

In [7]:
# property_type: First character uppercase -> lowercase
df_train['property_type'] = df_train['property_type'].str.lower()

# property_type: (hdb 2 rooms, hdb 3 rooms, hdb 4 rooms, hdb 5 rooms) -> hdb
temp = df_train['property_type'].str.startswith(('hdb 2', 'hdb 3', 'hdb 4', 'hdb 5'))
df_train['property_type'] = np.where((temp == True), 'hdb', df_train['property_type'])

# tenure: fill all hdb property_type with hdb defult tenure value - '99-year-leasehold'
hdb_tenure = df_train[(df_train['property_type'].str.startswith('hdb')) & df_train['tenure'].notna() ]['tenure'].unique()[0]
df_train['tenure'] = np.where((df_train['property_type'].str.startswith('hdb')) & (df_train['tenure'].isna()), hdb_tenure, df_train['tenure'])

# tenure: fill in NaN tenure with value from same address or property_name, otherwise fill in value from the nearest property in the same subzone
df_train.tenure = df_train.groupby('property_name').tenure.transform('first')
df_train.tenure = df_train.groupby('address').tenure.transform('first')

df_train_empty_tenure_filled = fill_NA_with_nearest_record(df_train, 'tenure')
df_train.loc[df_train.listing_id.isin(df_train_empty_tenure_filled.listing_id), ['tenure']] = df_train_empty_tenure_filled[['tenure']]


# built_year: fill in NaN built_year with value from same property_name or address, otherwise fill in with the nearest location record within the same subzone
df_train.built_year = df_train.groupby('property_name').built_year.transform('first')
df_train.built_year = df_train.groupby('address').built_year.transform('first')

df_train_empty_built_year_filled = fill_NA_with_nearest_record(df_train, 'built_year')
df_train.loc[df_train.listing_id.isin(df_train_empty_built_year_filled.listing_id), ['built_year']] = df_train_empty_built_year_filled[['built_year']]

df_train.head()

/var/folders/yd/59cvw_0n1x7b1_q6rcb9kcw9smn5lg/T/ipykernel_12757/378070349.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty[empty_feature] = df_empty.apply(lambda row: min_dist_feature_in_same_subzone(empty_feature, row['subzone'], row['lat'], row['lng'], df[(df[empty_feature].notna())]), axis=1)
/var/folders/yd/59cvw_0n1x7b1_q6rcb9kcw9smn5lg/T/ipykernel_12757/378070349.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty[empty_feature] = df_empty.apply(lambda row: min_dist_feature_in_sa

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
0,122881,hdb flat for sale in 866 yishun street 81,sembawang / yishun (d27),866 yishun street 81,hdb,99-year leasehold,1988.0,3.0,2.0,1115,...,unspecified,NaN,116.0,https://www.99.co/singapore/hdb/866-yishun-str...,1.414399,103.837196,0,yishun south,yishun,514500.0
1,259374,hdb flat for sale in 506b serangoon north aven...,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,99-year leasehold,1992.0,4.0,2.0,1575,...,unspecified,"1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbserangoon-e...,1.372597,103.875625,0,serangoon north,serangoon,995400.0
2,665422,4 bed condo for sale in meyerhouse,128 meyer road,meyerhouse,condo,freehold,2022.0,4.0,6.0,3070,...,partial,"studio, 3, 4, 5, 6 br",56.0,https://www.99.co/singapore/condos-apartments/...,1.298773,103.895798,0,mountbatten,marine parade,8485000.0
3,857699,3 bed condo for sale in leedon green,26 leedon heights,leedon green,condo,freehold,2023.0,3.0,2.0,958,...,partial,"studio, 1, 2, 3, 4 br",638.0,https://www.99.co/singapore/condos-apartments/...,1.312364,103.803271,0,farrer court,bukit timah,2626000.0
4,216061,2 bed condo for sale in one bernam,1 bernam street,one bernam,condo,99-year leasehold,2026.0,2.0,1.0,732,...,unspecified,"studio, 1, 2, 3, 4, 5 br",351.0,https://www.99.co/singapore/condos-apartments/...,1.273959,103.843635,0,anson,downtown core,1764000.0


In [8]:
# encode tenure
df_train = df_train.replace(["99-year leasehold", "110-year leasehold", "103-year leasehold", "102-year leasehold", "100-year leasehold"],[0,0,0,0,0])
df_train = df_train.replace(["999-year leasehold", "946-year leasehold", "956-year leasehold", "929-year leasehold", "947-year leasehold"],[1,1,1,1,1])
df_train = df_train.replace(["freehold"],[2])

In [9]:
# Can use encoding
import copy
df_train = copy.deepcopy(df_train)
df_train = df_train.replace(["good class bungalow", "hdb executive", "hdb"],[0,0,0])
df_train = df_train.replace(["cluster house", "executive condo", "walk-up", "terraced house", "semi-detached house", "landed", "townhouse", "corner terrace", "conservation house"],[1,1,1,1,1,1,1,1,1])
df_train = df_train.replace(["bungalow", "apartment", "condo", "shophouse", "land only"],[2,2,2,2,2])

In [10]:
# price: delete rows with price value 0
df_train = df_train[df_train['price'] != 0]
df_train[df_train['price'] == 0]

# drop num_baths empty rows (431)
df_train = df_train[df_train['num_baths'].notna()]

In [11]:
# drop 'listing_id', 'title', 'address', 'property_name', 'floor_level', 'furnishing', 'available_unit_types', 'total_num_units', 'property_details_url', 'elevation'
df_train.drop(columns=['listing_id', 'title', 'address', 'property_name', 'subzone', 'planning_area', 'floor_level', 'furnishing', 'available_unit_types', 'total_num_units', 'property_details_url', 'elevation'], inplace=True)
df_train.shape

(19719, 9)

In [ ]:
df_train.to_csv('./data/clean_data.csv', index=False)